In [1]:
import os
import wget
import object_detection
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
import tkinter as tk

from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg, NavigationToolbar2Tk)
# Implement the default Matplotlib key bindings.
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure
# from Tkinter import *
from PIL import ImageTk,Image
# import numpy as np
from pylab import cm
from numpy import asarray

In [2]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
CUSTOM_MODEL_NAME1 = 'my_ssd_mobnet1'
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'ANNOTATION_PATH1': os.path.join('Tensorflow', 'workspace','annotations1'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'CHECKPOINT_PATH1': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME1),
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'PIPELINE_CONFIG1':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME1, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME),
    'LABELMAP1': os.path.join(paths['ANNOTATION_PATH1'], LABEL_MAP_NAME)
}

In [3]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-3')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [4]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [5]:
# Load pipeline config and build a detection model
configs1 = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG1'])
detection_model1 = model_builder.build(model_config=configs1['model'], is_training=False)

# Restore checkpoint
ckpt1 = tf.compat.v2.train.Checkpoint(model=detection_model1)
ckpt1.restore(os.path.join(paths['CHECKPOINT_PATH1'], 'ckpt-3')).expect_partial()

@tf.function
def detect_fn1(image):
    image, shapes = detection_model1.preprocess(image)
    prediction_dict = detection_model1.predict(image, shapes)
    detections = detection_model1.postprocess(prediction_dict, shapes)
    return detections

In [6]:
category_index1 = label_map_util.create_category_index_from_labelmap(files['LABELMAP1'])

In [ ]:
# import cv2 
# import numpy as np
# from matplotlib import pyplot as plt
# %matplotlib inline
# from tkinter import *
# from tkinter import ttk
# from tkinter import filedialog

class Root(Tk):
    def __init__(self):
        super(Root, self).__init__()
        self.title("Upload")
        self.geometry("640x400")
        global lbg
        lbg = PhotoImage(file = '1.png')
        global logbg
        logbg = Canvas(self,width=1600,height=820)
        logbg.create_image(0,0,anchor = NW,image = lbg)
        logbg.grid(row=0,rowspan=50,column=0,columnspan=50)
#         self.wm_iconbitmap("icon.ico")
        
        self.labelFrame =tk.LabelFrame(self, text="Detect From Image")
        self.labelFrame.grid(column = 0, row = 1, padx=20, pady=20)
        self.button()
        
        self.labelFrame1 =tk.LabelFrame(self, text="Detect From Webcam")
        self.labelFrame1.grid(column = 0, row = 2, padx=30, pady=30)
        self.button1()
        
    def button(self):
        self.b3 = ttk.Button(self.labelFrame, text = "    Browse A File    ",command=self.fileDialog)
        self.b3.grid(column = 1, row = 1)
    
    def button1(self):
        self.b3 = ttk.Button(self.labelFrame1, text = "    Launch Webcam    ",command=self.webcam)
        self.b3.grid(column = 1, row = 1)
        
        
    def webcam(self):
        cap = cv2.VideoCapture(0)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        while cap.isOpened(): 
            ret, frame = cap.read()
            image_np = np.array(frame)

            input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
            detections = detect_fn1(input_tensor)

            num_detections = int(detections.pop('num_detections'))
            detections = {key: value[0, :num_detections].numpy()
                          for key, value in detections.items()}
            detections['num_detections'] = num_detections

            # detection_classes should be ints.
            detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

            label_id_offset = 1
            image_np_with_detections = image_np.copy()

            viz_utils.visualize_boxes_and_labels_on_image_array(
                        image_np_with_detections,
                        detections['detection_boxes'],
                        detections['detection_classes']+label_id_offset,
                        detections['detection_scores'],
                        category_index1,
                        use_normalized_coordinates=True,
                        max_boxes_to_draw=5,
                        min_score_thresh=.8,
                        agnostic_mode=False)

            cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))

            if cv2.waitKey(10) & 0xFF == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                break
    
    def show(self):
        global result
        result = Toplevel(root)
        result.title("object detection") 
        width = img1.width()
        height = img1.height()
        result.maxsize(width,height)
        result.minsize(300,300)
        bmibg = Canvas(result,width=1600,height=820)
        bmibg.create_image(0,0,anchor = NW,image = img1)
        bmibg.grid(row=0,rowspan=50,column=0,columnspan=50)
        
    def fileDialog(self):
        path = filedialog.askopenfilename(initialdir="C:/Users/sachi/project/TFODCourse/Tensorflow/workspace/images/test"
                                                    ,title="Select A File",filetype=(("jpeg","*.jpg"),("All Files","*.*")))
        img = Image.open(path)
        img = img.resize((720,600), Image.ANTIALIAS)
        img = asarray(img)
        
        
        image_np = np.array(img)

        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
        detections = detect_fn(input_tensor)

        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                      for key, value in detections.items()}
        detections['num_detections'] = num_detections

        # detection_classes should be ints.
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        label_id_offset = 1
        global image_np_with_detections
        image_np_with_detections = image_np.copy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
                    image_np_with_detections,
                    detections['detection_boxes'],
                    detections['detection_classes']+label_id_offset,
                    detections['detection_scores'],
                    category_index,
                    use_normalized_coordinates=True,
                    max_boxes_to_draw=5,
                    min_score_thresh=.8,
                    agnostic_mode=False)
        plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
        global fig
        fig =plt.figure()
#         ax.axis('off')
        global img1
        img1 =  ImageTk.PhotoImage(image=Image.fromarray(image_np_with_detections))
        canvas = tk.Canvas(root,width=800,height=500)
        self.show()
root=Root()
root.mainloop()